Record the time:
- convert each Sv and then open_mfdataset
- convert Sv and append

Record the time:
- combine inidividual daily Sv files into time windows
- make a big dataset and slice out time windows

Try to see if can see the to_zarr in parallel said here:
- https://xarray.pydata.org/en/stable/user-guide/io.html#appending-to-existing-zarr-stores

Keep the NaNs in the organized dataset.

In [1]:
from pathlib import Path
import datetime as dt

import xarray as xr

import echopype as ep
import gen_mvbs_utils

In [2]:
output_path = Path("./tmp_outputs/")
if not output_path.exists():
    output_path.mkdir()

In [3]:
ooi_CE04OSPS = (
    "https://rawdata.oceanobservatories.org/files/"
    "CE04OSPS/PC01B/ZPLSCB102_10.33.10.143/"
)

In [4]:
start = dt.datetime(year=2017, month=8, day=30)
end = dt.datetime(year=2017, month=8, day=30)

In [5]:
raw_file_list = gen_mvbs_utils.get_raw_file_url(
    file_url=ooi_CE04OSPS,
    start=start,
    end=end
)
len(raw_file_list)

15

## Convert individually and then combine

In [6]:
%time
for raw_file in raw_file_list:
    ed = ep.open_raw(raw_file=raw_file, sonar_model="EK60")
    ed.to_zarr(save_path=output_path, overwrite=True)
    ds_Sv = ep.calibrate.compute_Sv(ed)
    Sv_fname = Path(raw_file).with_name(Path(raw_file).stem + "_Sv")
    ds_Sv = ds_Sv.chunk({dim: ds_Sv[dim].size for dim in ds_Sv.dims})
    ds_Sv.to_zarr(output_path / Sv_fname.with_suffix(".zarr").name, mode="w")

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
17:56:04  parsing file OOI-D20170830-T000000.raw, time of first ping: 2017-Aug-30 00:00:00
17:56:09  saving tmp_outputs/OOI-D20170830-T000000.zarr
17:56:12  parsing file OOI-D20170830-T013905.raw, time of first ping: 2017-Aug-30 01:39:05
17:56:16  saving tmp_outputs/OOI-D20170830-T013905.zarr
17:56:20  parsing file OOI-D20170830-T031813.raw, time of first ping: 2017-Aug-30 03:18:13
17:56:24  saving tmp_outputs/OOI-D20170830-T031813.zarr
17:56:28  parsing file OOI-D20170830-T045714.raw, time of first ping: 2017-Aug-30 04:57:14
17:56:33  saving tmp_outputs/OOI-D20170830-T045714.zarr
17:56:36  parsing file OOI-D20170830-T063625.raw, time of first ping: 2017-Aug-30 06:36:25
17:56:40  saving tmp_outputs/OOI-D20170830-T063625.zarr
17:56:44  parsing file OOI-D20170830-T081527.raw, time of first ping: 2017-Aug-30 08:15:27
17:56:48  saving tmp_outputs/OOI-D20170830-T081527.zarr
17:56:52  parsing file OOI-D20170830-T095428.raw, time

In [7]:
Sv_zarr_list = list((output_path).glob("*_Sv.zarr"))

In [8]:
xr.open_dataset(Sv_zarr_list[0], engine="zarr")

<xarray.Dataset>
Dimensions:                (channel: 3, ping_time: 5924, range_sample: 1072,
                            filenames: 1, time3: 5924)
Coordinates:
  * channel                (channel) <U39 'GPT  38 kHz 00907208dd13 5-1 OOI.3...
  * ping_time              (ping_time) datetime64[ns] 2017-08-30T19:48:51.927...
  * range_sample           (range_sample) int64 0 1 2 3 ... 1068 1069 1070 1071
  * time3                  (time3) datetime64[ns] 2017-08-30T19:48:51.9270000...
Dimensions without coordinates: filenames
Data variables:
    Sv                     (channel, ping_time, range_sample) float64 ...
    echo_range             (channel, ping_time, range_sample) float64 ...
    equivalent_beam_angle  (channel, ping_time) float64 ...
    frequency_nominal      (channel) float64 ...
    gain_correction        (ping_time, channel) float64 ...
    sa_correction          (ping_time, channel) float64 ...
    sound_absorption       (channel, ping_time) float64 ...
    sound_speed            (channel, ping_time) float64 ...
    source_filenames       (filenames) <U120 ...
    water_level            (channel, time3) float64 ...
Attributes:
    processing_function:          calibrate.compute_Sv
    processing_software_name:     echopype
    processing_software_version:  0.6.0
    processing_time:              2022-06-28T00:57:51Z

In [9]:
ds_Sv_zarr_all = xr.open_mfdataset(
    Sv_zarr_list,
    engine="zarr",
    data_vars="minimal",
    coords="minimal",
    compat="override"  # this can be removed if filenames is a coordinate
)

/Users/wu-jung/miniconda3/envs/ooi-nmf-workflow-data-prep/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/Users/wu-jung/miniconda3/envs/ooi-nmf-workflow-data-prep/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ..

In [10]:
ds_Sv_zarr_all

<xarray.Dataset>
Dimensions:                (ping_time: 85795, channel: 3, range_sample: 1072,
                            filenames: 1, time3: 85795)
Coordinates:
  * ping_time              (ping_time) datetime64[ns] 2017-08-30T00:00:00.054...
  * channel                (channel) <U39 'GPT  38 kHz 00907208dd13 5-1 OOI.3...
  * range_sample           (range_sample) int64 0 1 2 3 ... 1068 1069 1070 1071
  * time3                  (time3) datetime64[ns] 2017-08-30T00:00:00.0540001...
Dimensions without coordinates: filenames
Data variables:
    Sv                     (channel, ping_time, range_sample) float64 dask.array<chunksize=(3, 85795, 1072), meta=np.ndarray>
    echo_range             (channel, ping_time, range_sample) float64 dask.array<chunksize=(3, 85795, 1072), meta=np.ndarray>
    equivalent_beam_angle  (channel, ping_time) float64 dask.array<chunksize=(3, 85795), meta=np.ndarray>
    frequency_nominal      (channel) float64 dask.array<chunksize=(3,), meta=np.ndarray>
    gain_correction        (ping_time, channel) float64 dask.array<chunksize=(85795, 3), meta=np.ndarray>
    sa_correction          (ping_time, channel) float64 dask.array<chunksize=(85795, 3), meta=np.ndarray>
    sound_absorption       (channel, ping_time) float64 dask.array<chunksize=(3, 85795), meta=np.ndarray>
    sound_speed            (channel, ping_time) float64 dask.array<chunksize=(3, 85795), meta=np.ndarray>
    source_filenames       (filenames) <U120 dask.array<chunksize=(1,), meta=np.ndarray>
    water_level            (channel, time3) float64 dask.array<chunksize=(3, 5923), meta=np.ndarray>
Attributes:
    processing_function:          calibrate.compute_Sv
    processing_software_name:     echopype
    processing_software_version:  0.6.0
    processing_time:              2022-06-28T00:56:11Z

## Try to append zarr file

In [11]:
ds_Sv["water_level"] = (
    ds_Sv["water_level"]
    .assign_coords({"ping_time": ("time3", ds_Sv["ping_time"].values)})
    .swap_dims({"time3": "ping_time"}).drop("time3")
)
# da.assign_coords({"lon_2": ("lon", lon_2)}))

In [12]:
%time
for seq, raw_file in enumerate(raw_file_list):
    ed = ep.open_raw(raw_file=raw_file, sonar_model="EK60")
    ed.to_zarr(save_path=output_path, overwrite=True)
    ds_Sv = ep.calibrate.compute_Sv(ed)
    # Change time3 to ping_time (the same for EK60)
    ds_Sv["water_level"] = (
        ds_Sv["water_level"]
        .assign_coords({"ping_time": ("time3", ds_Sv["ping_time"].values)})
        .swap_dims({"time3": "ping_time"}).drop("time3")
    )
    ds_Sv = ds_Sv.drop_dims("time3")
    # Assign chunk size
    ds_Sv = ds_Sv.chunk({dim: ds_Sv[dim].size for dim in ds_Sv.dims})
    if seq == 0:
        ds_Sv.to_zarr(output_path / "append_test.zarr", mode="w")
    else:
        ds_Sv.to_zarr(output_path / "append_test.zarr", mode="a", append_dim="ping_time")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 3.81 µs
17:58:08  parsing file OOI-D20170830-T000000.raw, time of first ping: 2017-Aug-30 00:00:00
17:58:12  overwriting tmp_outputs/OOI-D20170830-T000000.zarr
17:58:16  parsing file OOI-D20170830-T013905.raw, time of first ping: 2017-Aug-30 01:39:05
17:58:20  overwriting tmp_outputs/OOI-D20170830-T013905.zarr
17:58:24  parsing file OOI-D20170830-T031813.raw, time of first ping: 2017-Aug-30 03:18:13
17:58:28  overwriting tmp_outputs/OOI-D20170830-T031813.zarr
17:58:32  parsing file OOI-D20170830-T045714.raw, time of first ping: 2017-Aug-30 04:57:14
17:58:36  overwriting tmp_outputs/OOI-D20170830-T045714.zarr
17:58:40  parsing file OOI-D20170830-T063625.raw, time of first ping: 2017-Aug-30 06:36:25
17:58:44  overwriting tmp_outputs/OOI-D20170830-T063625.zarr
17:58:48  parsing file OOI-D20170830-T081527.raw, time of first ping: 2017-Aug-30 08:15:27
17:58:53  overwriting tmp_outputs/OOI-D20170830-T081527.zarr
17:58:56  parsing file O

In [13]:
ds_append = xr.open_dataset(output_path / "append_test.zarr", engine="zarr")

In [14]:
ds_append

<xarray.Dataset>
Dimensions:                (channel: 3, ping_time: 85795, range_sample: 1072,
                            filenames: 1)
Coordinates:
  * channel                (channel) <U39 'GPT  38 kHz 00907208dd13 5-1 OOI.3...
  * ping_time              (ping_time) datetime64[ns] 2017-08-30T00:00:00.054...
  * range_sample           (range_sample) int64 0 1 2 3 ... 1068 1069 1070 1071
Dimensions without coordinates: filenames
Data variables:
    Sv                     (channel, ping_time, range_sample) float64 ...
    echo_range             (channel, ping_time, range_sample) float64 ...
    equivalent_beam_angle  (channel, ping_time) float64 ...
    frequency_nominal      (channel) float64 ...
    gain_correction        (ping_time, channel) float64 ...
    sa_correction          (ping_time, channel) float64 ...
    sound_absorption       (channel, ping_time) float64 ...
    sound_speed            (channel, ping_time) float64 ...
    source_filenames       (filenames) <U120 ...
    water_level            (channel, ping_time) float64 ...
Attributes:
    processing_function:          calibrate.compute_Sv
    processing_software_name:     echopype
    processing_software_version:  0.6.0
    processing_time:              2022-06-28T01:00:07Z